# F29BC: Bio-Inspired Computation

- FFNN
- CNN
- RNN

## Feed-Forward Neural Network

Trains a simple deep NN on the MNIST dataset.

Gets to 98.40% test accuracy after 20 epochs
(there is **a lot** of margin for parameter tuning).

2 seconds per epoch on a K520 GPU.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.metrics import Precision, Recall

In [ ]:
batch_size = 128
num_classes = 10
epochs = 20

keras.utils.set_random_seed(7) # set `seed` for regeneration purposes.

In [ ]:
# loading data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 2s 0us/step
60000 train samples
10000 test samples


In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669706 (2.55 MB)
Trainable params: 669706 (2.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
precision = Precision()
recall = Recall()
# f_one_score = (2*precision*recall) / (precision+recall)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy', precision, recall])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 14s 12ms/step - loss: 0.2534 - accuracy: 0.9224 - precision: 0.9485 - recall: 0.9009 - val_loss: 0.0966 - val_accuracy: 0.9687 - val_precision: 0.9734 - val_recall: 0.9649
Epoch 2/20
469/469 [==============================] - 3s 7ms/step - loss: 0.1026 - accuracy: 0.9692 - precision: 0.9737 - recall: 0.9651 - val_loss: 0.0726 - val_accuracy: 0.9762 - val_precision: 0.9792 - val_recall: 0.9741
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0733 - accuracy: 0.9767 - precision: 0.9799 - recall: 0.9747 - val_loss: 0.0685 - val_accuracy: 0.9797 - val_precision: 0.9818 - val_recall: 0.9777
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0565 - accuracy: 0.9824 - precision: 0.9840 - recall: 0.9806 - val_loss: 0.0741 - val_accuracy: 0.9782 - val_precision: 0.9796 - val_recall: 0.9772
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0460 - accuracy: 0.9855 - 

- Difference between `model.evaluate()` and `model.predict()`: [here](https://stackoverflow.com/a/55049982/6306387)
- **`verbose`**: ? type of animation for showing training progress?

In [ ]:
score = model.evaluate(x_test, y_test, verbose='auto')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
pres = score[2]
recl = score[3]
f_one = (2*pres*recl) / (pres+recl)
print(f"Precision: {pres}, Recall: {recl}, F1-Score: {f_one}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0886 - accuracy: 0.9836 - precision: 0.9844 - recall: 0.9835
Test loss: 0.08860217034816742
Test accuracy: 0.9836000204086304
Precision: 0.9843859672546387, Recall: 0.9835000038146973, F1-Score: 0.9839427860995309


- Another way to evaluate: getting model predictions and using sci-kit learn's `classification report`:

In [ ]:
from sklearn.metrics import classification_report as eval_metrics
from numpy import argmax

y_pred =  argmax(model.predict(x_test), axis=1)
print(eval_metrics(y_test, y_pred))

313/313 [==============================] - 1s 2ms/step


ValueError: ignored

## Convolutional Neural Networks

Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs

There is still a lot of margin for parameter tuning.

16 seconds per epoch on a GRID K520 GPU.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


## LSTM Text Generator

In [ ]:
'''
#Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
# from keras.utils.data_utils import get_file
from keras.utils import get_file  # data_utils is no longer available for newer version of keras
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
    #for diversity in [0.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

on_epoch_end(0, 0)

model.fit(x, y,
          batch_size=128,
          epochs=1,
          callbacks=[print_callback])
